# A Way-too-quick Intro to Astropy and Astroquery

Welcome! This notebook will walk you through a few things that you can do with the [Astropy] and [Astroquery] Python modules. We only have an hour, so this will only scratch the surface.

[Astropy]: https://www.astropy.org/
[Astroquery]: https://astroquery.readthedocs.io/

**Astropy** is a library of all sorts of Python code that is useful for astronomers. It's very wide-ranging so it's worth your while to skim [the documentation] to get a sense of all the things that it can do! We'll just look at a few foundational parts of it.

It's worth mentioning that Astropy is very carefully designed, and it is highly-respected across the Python world. We are lucky to have it!

[the documentation]: https://astroquery.readthedocs.io/en/latest/

**Astroquery** is a related package that helps you query astronomy databases and download data from telescope archives. It is super useful since so much astronomy data is in the cloud these days.

## Preliminaries

As is almost always the case, we will need to start off our notebook by importing various modules that we'll need.

In [ ]:
import numpy as np
from astropy import units as u
from astropy.timeseries import TimeSeries
from aas_timeseries import InteractiveTimeSeriesFigure
from astroquery.mast import Observations
# You can safely ignore the "AstropyDeprecationWarning".

## Astropy "Quantities" and Units

Astropy has a very neat system for tagging your variables with what units they're measured in. This is great since confusion over units is a **constant** source of bugs in scientific software.

Here we'll set up a variable called `reff`, for "effective radius", that is measured in parsecs. The big trick is that you multiply your number by a special unit variable:

In [ ]:
reff = 29 * u.pc
reff # this pattern creates the variable, and then prints it out so we can see what we got

What is that in meters?

In [ ]:
reff.to(u.m)

#### Exercise!

`reff` is the effective *radius* of a galaxy. Pretending the galaxy is a circle, what is its effective area in square kilometers? You can just type in π as a number, or use the variable `np.pi`.


In [ ]:
# Use Python as a calculator here to compute an answer! Preferably using the `reff.to()` function.

The Astropy units system forces you to specify your units whenever there's a chance of doing something ambiguous.

In [ ]:
reff + 14  # This will issue an error, and rightfully so. Did you mean 14 km? 14 pc? 14 feet??

Astropy includes physical constants with their units tagged, which helps make sure you get your equations right. For instance, let's write a function to calculate the spectral radiance of a blackbody in the Rayleigh-Jeans limit (no worries if you're not familiar with the physics — we're just multiplying a few numbers):

In [ ]:
def rj_blackbody(temperature, frequency):
    from astropy.constants import c, k_B
    
    if not u.Quantity(temperature).unit.is_equivalent(u.K):
        raise ValueError('temperature must be in a Kelvin-equivalent unit')
    
    if not u.Quantity(frequency).unit.is_equivalent(u.Hz):
        raise ValueError('temperature must be in a Hertz-equivalent unit')
    
    return 2 * frequency**2 * k_B * temperature / c**2

The code to this function was short, but using the units system makes sure that we're always clear on what we're talking about. What are the units of spectral radiance again, anyway?

In [ ]:
spec_rad = rj_blackbody(5800 * u.K, 10 * u.GHz)
spec_rad.decompose()

The units subsystem is just one tiny piece of Astropy, but it's so useful that almost every other part of Astropy builds on top of it.

## Astroquery: Downloading TESS data

We'll change gears now and download some data from NASA using Astroquery!

At the top, we imported a variable named `Observations` from the `astroquery.mast` module, which sends queries to the "MAST" archive run by NASA. MAST mainly contains data from Hubble, but it stores data for lots of other missions too.

First, we'll ask MAST what observations it has of the star HD 21749:

In [ ]:
obslist = Observations.query_object("HD 21749", radius="0.02 deg")
obslist  # once again, ending the cell with the variable name will show us what it contains

The way MAST organizes things, the next step is to download a list of "data products". We'll limit ourselves to ones from the recent TESS mission: 

In [ ]:
is_tess_obs = (obslist['obs_collection'] == 'TESS')
dp = Observations.get_product_list(obslist[is_tess_obs])
dp

Now we'll zero in on the "lightcurve" data products in particular and have Astroquery download them for us:

In [ ]:
is_lightcurve_product = (dp['dataproduct_type'] == 'timeseries') & (dp['productSubGroupDescription'] == 'LC')
manifest = Observations.download_products(dp[is_lightcurve_product])
manifest  # a data table listing the files that were downloaded

Since lightcurve data pop up all over astronomy, Astropy has special tools for dealing with them. We can read in the first data file into a `TimeSeries` object like so:

In [ ]:
ts = TimeSeries.read(manifest['Local Path'][0], format='tess.fits')
ts

What's the first thing to do with a data set? Plot it! This code will set up a plot of the light curve:

In [ ]:
fig = InteractiveTimeSeriesFigure()
fig.xlabel = 'Time (UTC)'
fig.ylabel = 'Flux (electron/s)'
markers = fig.add_markers(time_series=ts, column='sap_flux', label='SAP Flux', size=10)

And this code will show it:

In [ ]:
fig.preview_interactive()

On my computer, holding down the `Alt` key and then drawing a box with my mouse will zoom in on the box I draw. Try exploring the different dips and see what their shape is like. What do you think could cause such a transit shape?

## If we have time: Astropy Coordinates and Images

Sorry, less commentary here! These cells will walk through Astropy's tools for working with sky coordinates and plotting images.

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.table import Table
from IPython.display import Image
from urllib.parse import urlencode
from urllib.request import urlretrieve

# Set up matplotlib and use a nicer set of plot parameters
from astropy.visualization import astropy_mpl_style
import matplotlib.pyplot as plt
plt.style.use(astropy_mpl_style)
%matplotlib inline

In [ ]:
# initialize a SkyCood object named hcg7_center at the location of Hickson Compact Group 7
hcg7_center = SkyCoord.from_name('HCG 7')

In [ ]:
print(hcg7_center.ra, hcg7_center.dec)
print(hcg7_center.ra.hour, hcg7_center.dec.deg)

Well, this is kind of lame. I've been singing the praises of Astroquery, but the code I'm copy/pasting doesn't use it :-(

In [ ]:
# tell the SDSS service how big of a cutout we want
im_size = 12 * u.arcmin  # get a 12 arcmin square
im_pixels = 1024
cutoutbaseurl = 'http://skyservice.pha.jhu.edu/DR12/ImgCutout/getjpeg.aspx'
query_string = urlencode(dict(ra=hcg7_center.ra.deg,
                              dec=hcg7_center.dec.deg,
                              width=im_pixels, height=im_pixels,
                              scale=im_size.to(u.arcsec).value/im_pixels))
url = cutoutbaseurl + '?' + query_string

# this downloads the image to your disk
urlretrieve(url, 'HCG7_SDSS_cutout.jpg')

In [ ]:
Image('HCG7_SDSS_cutout.jpg')

#### Exercise

Emulate the steps above to download an image of another interesting object on the sky. If you can't think of anything in particular, try a Messier object: "M1", "M31", "M101" ... just "M" followed by a small number.

In [ ]:
# here we go!